# Machine Learning Portfolio 2

|Name|Github|Kaggle|
|----|------|------|
|Henry Lau|HenryLau08|Henry Lau|
|Mohamed Belaachir|mobelaachir|Mo Belaachir|
|Jayden Debi-Tewari|Jaydendt1|jaydendt123|
|Quincy Soerohardjo|quincysoerohardjo2002|Quincy Soerohardjo|
|Mattias Aareleid|mattyonaize|Mattias Aareleid|

## Table of Contents
- [Data Overview](#data-overview)
- [Exploratory Data Analysis](#exploratory-data-analysis)
- [Modeling](#modeling)

- [Results](#results)
- [Conclusion & Advice](#conclusion--advice)
- [Sources](#sources)

In [8]:
# Libraries

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.fft import fft, ifft
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score


ModuleNotFoundError: No module named 'numpy'

## Data Overview

In [5]:
train = pd.read_csv('train.csv', index_col='date_hour')
test = pd.read_csv('test.csv', index_col='date_hour')
sample_submission = pd.read_csv('sample_submission.csv')

In [ ]:
# Train and Test data
display(train.head(10), test.head(10))

,holiday,weathersit,temp,atemp,hum,windspeed,cnt
date_hour,,,,,,,
2011-01-01 00:00:00,0,1,0.24,0.2879,0.81,0.0000,16
2011-01-01 01:00:00,0,1,0.22,0.2727,0.80,0.0000,40
2011-01-01 02:00:00,0,1,0.22,0.2727,0.80,0.0000,32
2011-01-01 03:00:00,0,1,0.24,0.2879,0.75,0.0000,13
2011-01-01 04:00:00,0,1,0.24,0.2879,0.75,0.0000,1
2011-01-01 05:00:00,0,2,0.24,0.2576,0.75,0.0896,1
2011-01-01 06:00:00,0,1,0.22,0.2727,0.80,0.0000,2
2011-01-01 07:00:00,0,1,0.20,0.2576,0.86,0.0000,3
2011-01-01 08:00:00,0,1,0.24,0.2879,0.75,0.0000,8


,holiday,weathersit,temp,atemp,hum,windspeed
date_hour,,,,,,
2012-12-01 00:00:00,0,1,0.26,0.3030,0.81,0.0000
2012-12-01 01:00:00,0,1,0.26,0.3030,0.81,0.0000
2012-12-01 02:00:00,0,2,0.26,0.3030,0.81,0.0000
2012-12-01 03:00:00,0,2,0.26,0.2727,0.81,0.1343
2012-12-01 04:00:00,0,1,0.26,0.2879,0.81,0.0896
2012-12-01 05:00:00,0,1,0.24,0.2576,0.87,0.0896
2012-12-01 06:00:00,0,1,0.24,0.2424,0.87,0.1343
2012-12-01 07:00:00,0,2,0.24,0.2424,0.87,0.1343
2012-12-01 08:00:00,0,2,0.24,0.2424,0.87,0.1343


In [8]:
# Sample submission for kaggle
display(sample_submission.head(10))

,date_hour,cnt
0,2012-12-01 00:00:00,784
1,2012-12-01 01:00:00,80
2,2012-12-01 02:00:00,605
3,2012-12-01 03:00:00,604
4,2012-12-01 04:00:00,544
5,2012-12-01 05:00:00,100
6,2012-12-01 06:00:00,344
7,2012-12-01 07:00:00,18
8,2012-12-01 08:00:00,57
9,2012-12-01 09:00:00,603


## Exploratory Data Analysis

## Feature Engineering

Seizoenspatronen met Fourier analyse

### Wat is Fourier analyse?
Het is een wiskundig methode dat patronen vindt binnen een genormaliseerde timeseries dataset. Het vereenvoudigt complexe data door het te veranderen naar een serie van trigonomische of exponentiele functies. Door alle complicerende factors van het dataset weg te halen kunnen patronen makkelijker herkend worden, waardoor voorspellingen maken makkelijker wordt.

[(Hayes, 2023)](https://www.investopedia.com/terms/f/fourieranalysis.asp) 

De formule van een Fourier series:
$$
f(t) = \frac{a_0}{2} + \sum_{k=1}^{\infty}(a_k\cos(2\pi kt) + b_k\sin(2\pi kt))
$$

waar
- $\frac{a_0}{2}$ het constante term representeert,
- $a_k\cos(2\pi kt)$ en $b_k\sin(2\pi kt)$ de cosinus en sinus termen zijn,
- $k$ het harmonische frequentie is,
- $\sum_{k=1}^{\infty}$ zegt dat er een oneindig serie van sinus en cosinus functies met verschillende frequenties en amplitudes opgeteld worden, die $f(t)$ preciezer zou vinden

Een fourier analyse is het manier waarop functies benaderd kunnen worden door het combinatie van trigonometrische functies. 

[(3Blue1Brown, 2018)](https://www.youtube.com/watch?v=spUNpyF58BY )


In de visualisaties is het te zien dat de target variabele begint toe te nemen rond april, en afneemt rond de wintermaanden, beginnend met november.

Onderzoek naar trends

Na een Fourier analyse uit te voeren op de data, is het te zien dat de grootste trends jaarlijks en dagelijks zijn. De jaarlijkse trends zou in dit geval groei van het populariteit van het product over tijd kunnen zijn, dit ligt aan het feit dat de jaarlijkse piek in 2011 lager is dan die van 2012.

De dagelijkse trend kan liggen aan andere variabelen, zoals temperatuur, weeromstandigheden of vakantie.

## Modeling

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [ ]:
X_train = ts_fe.X_train.copy()
y_train = ts_fe.y_train.copy()

### K-Nearest Neighbors Regression

De KNN algoritme is een techniek die zich richt op de waardes rond een bepaald punt. Die punt wordt dan geclassificeerd op basis van het aantal ingestelde 'nearest neighbors', oftewel naaste buren. Als je dan instelt dat je de 5 'nearest neighbors' kiest, dan wordt de waarde geclassificeerd als de meerderheid van die buren. 

[(IBM, 2024)](https://www.ibm.com/topics/knn)

KNN Regression is een eenvoudig algoritme dat gebaseerd is op het afstand van een punt tot een bepaald aantal dichtbijzijnde punten (K). Het afstand kan op een aantal verschillende manieren gemeten worden. Ten opzichte van KNN-classificatie, in KNN-regressie is de voorspelde waarde van de datapunt gelijk aan de gemiddelde van de dichtbijzijnde punten 
(target-waardes).

[(Singh, 2024)](https://www.analyticsvidhya.com/blog/2018/08/k-nearest-neighbor-introduction-regression-python/#How_Does_the_KNN_Algorithm_Work?)

De afstand naar de gekozen punt wordt meestal berekent met de manhattan- of euclidean-methode. De **Euclidean** methode berekent een rechte lijn tussen de gekozen punt en een 'buur'. Euclidean afstand bereken je met de volgende formule:

$$
d(x, y) = \sqrt{\sum_{i=1}^{n} (y_i - x_i)^2}
$$

De **Manhattan** methode kiest voor het meten van de absolute waarde van het verschil tussen twee punten. De naam komt van de visualisatie die vaak met dit methode komt, omdat het als een rooster uitziet, net alsof je Manhattan van boven ziet. De Manhattan afstand kun je berekenen met dit methode:

$$
d(x, y) = (\sum_{i=1}^{m} |x_i - y_i|)
$$

[(IBM, 2024)](https://www.ibm.com/topics/knn)

**Standardisatie**

De manier waarop KNN beslissingen neemt heeft veel te maken met het afstand tussen waardes, waardoor de schaal veel invloed heeft op de afstandsbepaling. Door de waardes te standardiseren, voorkom je dat sommige features meer voorkomen.

[(Goedegebuure, 2021)](https://bookdown.org/robert_statmind/mmsc_test_01/the-k-nn-algorithm.html#standardizing-data)

**Hyperparameters**

De belangrijkste hyperparameter bij KNN is de het aantal buren (k). Een grotere k-waarde geeft een eenvoudige decision-boundary en vermindert overfitting, maar is niet handig met alle datasets. Een kleinere k-waarde kan er juist voor zorgen dat er sprake is van overfitting. 

[(Abdallah, 2023)](https://www.linkedin.com/pulse/improve-model-hyperparameter-tuning-k-nearest-muctary-abdallah-1e)


In [ ]:
# Daadwerkelijke KNN model

knn_model = KNeighborsRegressor(n_neighbors=3, weights='distance', metric='manhattan')
knn_model.fit(X_train, y_train)

knn_score = cross_val_score(knn_model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=10, n_jobs=-1, verbose=1).mean()
print(f"KNN score: {knn_score}")

In [ ]:
# Beste parameters

knn_param = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_model = KNeighborsRegressor()
grid_search = GridSearchCV(knn_model, knn_param, cv=10, scoring='neg_root_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
print(f"Beste parameters: {grid_search.best_params_}")

knn_tuned = grid_search.best_estimator_
knn_score = cross_val_score(knn_model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=10, n_jobs=-1, verbose=1).mean()
print(f"KNN score: {knn_score}")

### Support Vector Regression

Een SVR model heeft als doel het voorspellen van een continu waarde, waar de waarde binnen de tolerantie-grens ϵ moet liggen van de correcte waarde. Alle voorspellingen die buiten dit grens liggen worden gepenaliseerd.

De SVR model kan ook non-lineaire relaties vinden met zijn **kernel-trick functie**. Hiermee krijgt de model een extra dimensie dat complexe relaties kan onderscheiden.

[(Sethi, 2024)](https://www.analyticsvidhya.com/blog/2020/03/support-vector-regression-tutorial-for-machine-learning/)

### SVR model bij een lineaire relatie

$$
w*x+b = y
$$

- $w$ : vector die de helling van de lijn of hypervlak bepaalt
- $x$ : invoerpunt
- $b$ : bias-term, oftewel het afstand vanaf het oorsprong langs het normaalvector $w$
- $y$ : de optimale hypervlak

Een SVR wil in dit geval aan het conditie $-a < y-wx+b < a$ te voldoen, waar $a$ het beste lijn binnen de grenswaarde is. De punten binnen de grens worden gebruikt om de waarde te voorspellen.

[(Gurucharan, 2020)](https://towardsdatascience.com/machine-learning-basics-support-vector-regression-660306ac5226)

### Afstand en de tolerantie-grens ϵ

$$
d_{i} = |y_i - (w^tx_i + b)|
$$

- $d_{i}$ : de absolute afwijking tussen de voorspelling en de daadwerkelijke waarde
- ϵ : de tolerantie-grens, oftewel de maximaal toegestane afwijking dat nog goed wordt gekeurd/geen penalty krijgt

Voor alle punten waar $d_i$ > ϵ geldt, wordt er een penalty gegeven. De penalty wordt gegeven volgens de **loss-functie**.

### Kernel trick

De kernel-trick is een techniek dat gebruikt wordt bij het toepassen van de SVR algoritme, dat niet-lineair-verdeelbare data neemt en dat omzet tot in een hoger dimensie waar de data wel lineair verdeelbaar is.

[(Yadav, 2023)](https://medium.com/@Suraj_Yadav/what-is-kernel-trick-in-svm-interview-questions-related-to-kernel-trick-97674401c48d)

### Loss functie

De loss functie van SVR wordt de lineair epsilon-insensitive loss genoemd. Hier is de loss gemeten door het afstand te bepalen tussen de gekozen waarde $y$ en de $\epsilon$ tolerantiegrens. De wiskundige notatie van de loss function:

$$
L_{\epsilon} =
\begin{cases} 0, & \text{als } |y - f(x)| \leq \epsilon \\
|y - f(x)| - \epsilon, & \text{anders} \end{cases}
$$

- $y$ : De ware classificatie voor de datapunt
- $f(x)$ : Het voorspelde waarde van de datapunt

[(Mathworks, z.d.)](https://nl.mathworks.com/help/stats/understanding-support-vector-machine-regression.html)

### Regularisatie

De manier van regularisatie in de Support Vector Regression werkt voornamelijk met het regularisatie parameter C. Dit parameter controleert de afwisseling tussen het negatieve correlatie met het maximaliseren van de marge en het minimaliseren van fouten bij het voorspelling. 

**Hyperparameters**
- C, het regularisatie parameter
- epsilon/$\epsilon$, de tolerantie-grens 
- gamma, het kernel-coefficient
- kernel, de keuze van kernel

[(Van Otten, 2024)](https://spotintelligence.com/2024/05/08/support-vector-regression-svr/)

In [ ]:
# Daadwerkelijke model

svm_model = SVR(C=100, epsilon=0.001, gamma=0.1, kernel='rbf')
svm_model.fit(X_train, y_train)

svm_score = cross_val_score(svm_model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=10, n_jobs=-1, verbose=1).mean()
print(f"SVM score: {svm_score}")

In [7]:
# Beste parameters

param_grid = {
    'kernel': ['rbf'],
    'C': [0.01, 0.1, 1, 10, 100],
    'epsilon': [0.001, 0.01, 0.1, 1],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1, 10, 100],
}

svm_model = SVR()
svmgrid = GridSearchCV(svm_model, param_grid, cv=5, scoring='neg_root_mean_squared_error', verbose=1, n_jobs=-1)
svmgrid.fit(X_train, y_train)
print(f"Beste parameters: {svmgrid.best_params_}")

svm_tuned = grid_search.best_estimator_
svm_score = cross_val_score(svm_tuned, X_train, y_train, scoring='neg_root_mean_squared_error', cv=10, n_jobs=-1, verbose=1).mean()
print(f"SVM score: {svm_score}")

NameError: name 'SVR' is not defined

## Results

### Overview

#### KNN
De KNN Regressor is heel snel, maar scoort niet goed met de dataset, zelfs met optimale hyperparameters. Dit komt waarschijnlijk omdat het model best eenvoudig werkt, waardoor het niet goed omgaat met time-series data. 

#### Linear Regression
Het model heeft een snelle runtijd, maar de scoren zijn niet goed. Na het toepassen van grid search met verschillende hyperparameters blijft de score bijna hetzelfde. Ook het gebruik van regularisatie heeft niet veel geholpen om de score van het model te verbeteren.

#### Decision Tree
Het model heeft een hele korte runtijd en scoort overigens ook best goed. Optimale hyperparameters met behulp van grid search gevonden. Met random forest zou het mogelijk zijn om de voorspellingen nauwkeuriger te maken en de score te verbeteren.

#### SVR
De linear en poly kernels leiden tot een hele lange runtijd, dus het vinden van optimale hyperparameters bij deze kernels met grid search zou onmogelijk lang duren. Verder, door het ontwerp van SVR is het ook niet praktisch om met grote (time-series) datasets te werken. Hierdoor heeft de model laag gescoord.

#### Random Forest
Het model neemt lang door de optimale hyperparameters van grid search, maar heeft een beter score gehaald dan de enkele Decision Tree. 

#### Gradient Boost
Het neemt werkelijk meer tijd om te trainen. Na een keer runnen, overgestapt naar XGBoost en bepaalde parameters voor gridsearch aangepast bij Gradient Boost.

#### XGBoost
Na een keer gridsearch runnen, dan handmatig wat waardes voor de parameters testen die niet voor gridsearch werd gebruikt. Kwam wat parameters tegen die het model een klein beetje beter heeft kunnen maken.

#### AdaBoost
Na een keer gridsearch runnen, ook handmatig getest en weer gridsearch. De score is niet zo goed als van XGBoost. Het kan zijn doordat hoe Ada werkt met moeilijkere voorspellingen.

#### Stacking

#### SARIMA(X)

#### Prophet

#### Hybrid


De Root Mean Squared Error (RMSE) scores worden in dit tabel genoteerd. Als er betere scores zijn op kaggle, dan komt het doordat de submission gedaan werd na het inleveren van de opdracht en/of er veranderingen zijn gebracht in feature engineering en niet genoeg tijd over is om alle modellen te hertrainen.

|Model              |Notebook Score|Kaggle Score|
|-------------------|--------------|------------|
|KNN                |147.76        |0           |
|Logistic Regression|125.42        |0           |
|Decision Tree      |0             |0           |
|SVR                |138.88        |0           |
|Random Forest      |0             |0           |
|Gradient Boost     |52.88         |0           |
|XGBoost            |50.71         |0           |
|AdaBoost           |103.00        |0           |
|Stacking           |54.38         |0           |
|SARIMA(X)          |0             |0           |
|Prophet            |0             |0           |
|hybride model      |0             |0           |

## Conclusion & Advice

## Sources

- 3Blue1Brown. (2018, 26 januari). But what is the Fourier Transform?  A visual introduction. [Video]. YouTube. https://www.youtube.com/watch?v=spUNpyF58BY

- Abdallah, M. (2023, 2 mei). Improve model - Hyperparameter tuning in k-nearest neighbors. https://www.linkedin.com/pulse/improve-model-hyperparameter-tuning-k-nearest-muctary-abdallah-1e

- Goedegebuure, R. (2021, 8 februari). Chapter 3 The K-NN Algorithm | Market Segmentation & Clustering. https://bookdown.org/robert_statmind/mmsc_test_01/the-k-nn-algorithm.html#standardizing-data

- Gurucharan, M. K. (2021, 15 december). Machine Learning Basics: support vector regression - towards data science. Medium. https://towardsdatascience.com/machine-learning-basics-support-vector-regression-660306ac5226

- Hayes, A. (2023, 4 juli). Fourier Analysis: What it Means, How it Works. Investopedia. https://www.investopedia.com/terms/f/fourieranalysis.asp

- IBM. (2024, 28 oktober). KNN. IBM. https://www.ibm.com/topics/knn

- Mathworks. (z.d.). Understanding support vector machine regression. https://nl.mathworks.com/help/stats/understanding-support-vector-machine-regression.html

- Sethi, A. (2024, 20 november). Support Vector Regression Tutorial for Machine Learning. Analytics Vidhya. https://www.analyticsvidhya.com/blog/2020/03/support-vector-regression-tutorial-for-machine-learning/

- Singh, A. (2024, 8 oktober). KNN algorithm: Introduction to K-Nearest Neighbors Algorithm for Regression. Analytics Vidhya. https://www.analyticsvidhya.com/blog/2018/08/k-nearest-neighbor-introduction-regression-python/#How_Does_the_KNN_Algorithm_Work?

- Van Otten, N. (2024, 11 oktober). Support Vector Regression (SVR) Simplified & How To Tutorial In Python. Spot Intelligence. https://spotintelligence.com/2024/05/08/support-vector-regression-svr/

- Yadav, S. (2023, 30 april). What is Kernel Trick in SVM ? Interview questions related to Kernel Trick. Medium. https://medium.com/@Suraj_Yadav/what-is-kernel-trick-in-svm-interview-questions-related-to-kernel-trick-97674401c48d
